# Python을 이용하여 선형대수를 공부해봅시다

##### 1.  유한체 (GF(2))에서의 RREF 연산 

In [7]:
import numpy as np
import pandas as pd


def rref_gf2(A, return_ops=False, as_bool=False,
             track_transpose=False, track_right=False):
    """
    GF(2) RREF with history H.
    - Always returns R, H, pivots, and optionally T (A^T after same row ops),
      and A_right (A after same *column* ops).

    Parameters
    ----------
    A : array_like (m, n)
        Input over GF(2). Any integers are reduced mod 2.
    return_ops : bool
        If True, also return the list of row-ops ("swap", i, j) or ("add", dst, src).
    as_bool : bool
        If True, return bool arrays instead of uint8.
    track_transpose : bool
        If True, apply the *same row ops* to A.T as well.
        Requires A to be square (n x n).
    track_right : bool
        If True, also return A_right = A @ H.T (i.e., same ops as *column* ops on A).

    Returns
    -------
    R : (m, n)  RREF(A) over GF(2)
    H : (m, m)  Transform so that (H @ A) % 2 == R
    pivots : list[(row, col)]
    T : (n, m) or None
        A.T after the same row ops: (H @ A.T) % 2 (requires square A).
    A_right : (m, n) or None
        A after the corresponding column ops: (A @ H.T) % 2
    ops : list, optional
    """
    R = (np.array(A, dtype=np.uint8) & 1).copy()
    m, n = R.shape
    H = np.eye(m, dtype=np.uint8)
    T = None
    A_right = None

    if track_transpose:
        if m != n:
            raise ValueError("track_transpose=True면 A는 정사각(n×n)이어야 해.")
        T = R.T.copy()

    pivots, ops = [], []
    row = 0
    for col in range(n):
        if row >= m:
            break
        # pivot search
        cand = np.flatnonzero(R[row:, col]) + row
        if cand.size == 0:
            continue
        p = int(cand[0])

        # swap rows
        if p != row:
            R[[row, p]] = R[[p, row]]
            H[[row, p]] = H[[p, row]]
            if track_transpose:
                T[[row, p]] = T[[p, row]]
            ops.append(("swap", row, p))

        # eliminate all other 1s in this column (above & below)
        ones = np.flatnonzero(R[:, col])
        ones = ones[ones != row]
        if ones.size:
            R[ones, :] ^= R[row, :]
            H[ones, :] ^= H[row, :]
            if track_transpose:
                T[ones, :] ^= T[row, :]
            for i in ones:
                ops.append(("add", int(i), row))

        pivots.append((row, col))
        row += 1

    if track_right:
        A0 = (np.array(A, dtype=np.uint8) & 1)
        A_right = (A0 @ H.T) % 2

    if as_bool:
        R = R.astype(bool); H = H.astype(bool)
        if T is not None: T = T.astype(bool)
        if A_right is not None: A_right = A_right.astype(bool)

    if return_ops:
        return R, H, pivots, T, A_right, ops
    return R, H, pivots, T, A_right


In [3]:
A = np.array([
    [1,0,1,1],
    [1,1,0,1],
    [0,1,1,0],
    [1,0,0,1]
], dtype=np.uint8)

R, H, piv, T, A_right, ops = rref_gf2(A, return_ops=True,
                                      track_transpose=True, track_right=True)

# 검증
ok_R = np.array_equal((H @ A) % 2, R)
ok_T = np.array_equal((H @ A.T) % 2, T)   # 전치에 같은 '행 연산'
ok_C = np.array_equal((A @ H.T) % 2, A_right)  # 같은 '열 연산'

print("OK left (H@A==R)?", ok_R)
print("OK transpose (H@A.T==T)?", ok_T)
print("OK right (A@H.T==A_right)?", ok_C)
print("pivots:", piv)
print("ops:", ops)
print("R=\n", R)
#print("T (A^T after ops)=\n", T)
print("A_right (A after column-ops)=\n", A_right)


OK left (H@A==R)? True
OK transpose (H@A.T==T)? True
OK right (A@H.T==A_right)? True
pivots: [(0, 0), (1, 1), (2, 2)]
ops: [('add', 1, 0), ('add', 3, 0), ('add', 2, 1), ('swap', 2, 3), ('add', 0, 2), ('add', 1, 2)]
R=
 [[1 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 0]]
A_right (A after column-ops)=
 [[1 1 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 1 0 1]]


##### Return value R이 곧 RREF결과값이며, A_right는 RREF 연산을 기록한 History Matrix임

In [34]:
np.sqrt(4)
import math
math.sqrt(4)
import sympy
sympy.sqrt(4)

2

In [39]:
df = pd.read_csv('hamming_7x30_codewords.csv')

df.drop('name', axis = 1, inplace=True)

In [40]:
df

,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,...,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30
0,0,0,0,1,1,1,1,1,0,1,...,1,1,1,1,0,1,1,0,1,0
1,1,1,0,1,0,0,1,0,1,1,...,0,1,1,0,1,0,1,1,0,0
2,1,0,1,1,1,0,0,0,1,0,...,0,0,0,1,1,1,1,0,1,0
3,0,1,1,1,0,1,1,0,0,1,...,1,0,1,0,0,0,1,1,1,0
4,1,0,1,1,1,0,1,1,1,1,...,0,0,1,1,1,1,1,0,0,0
5,1,1,0,1,0,0,0,1,1,0,...,0,1,0,0,1,0,1,1,1,0
6,0,0,0,1,1,1,0,0,0,0,...,1,1,0,1,0,1,1,0,0,0
